Code

In [30]:
%%writefile app.py

Overwriting app.py


In [36]:
#imports
import streamlit as st
from typing import List, Dict
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from google.colab import userdata
userdata.get('GEN_API_KEY')
import urllib
print("Password for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.148.8.246


In [34]:
def get_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader= PdfReader(pdf)
        for page in pdf_reader.pages:
            text+= page.extract_text()
    return  text



def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks


def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")


def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, ensure that you provide all the details, if the answer is not in
    provided context just say, "Gomen.. but could you run that by me again?", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro", client="genai", temperature=0.3)

    prompt = PromptTemplate(template=prompt_template,
                            input_variables=["context", "question"])
    contents: List[Dict[str, Dict[str, str]]] = [
  {"role": "user", "parts": {"text": "Hey!"}},
  {"role": "model", "parts": {"text": "Ayy.. Ready for it?"}},
  {"role": "user", "parts": {"text": "Wow! You are really helping."}}, {"role": "model", "parts": {"text": "Arigatou! I'm Happy to hear that."}}
]
    chain = load_qa_chain(llm=model, chain_type="stuff", prompt=prompt)
    return chain

def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

    new_db = FAISS.load_local("faiss_index", embeddings)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()


    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    st.write("Reply: ", response["output_text"])




def main():
    st.set_page_config("BubbleBanter")
    st.header("Banter with Gemini!")

    user_question = st.text_input("You can ask anything from the pdf(s)!")

    if user_question:
        user_input(user_question)

    with st.sidebar:
        st.title("Menu:")
        pdf_docs = st.file_uploader("Upload the files and click Bubble Up button. Rest is upto Gemini.", accept_multiple_files=True)
        if st.button("Bubble Up"):
            with st.spinner("Gemini is busy going through the files..."):
                raw_text = get_pdf_text(pdf_docs)
                text_chunks = get_text_chunks(raw_text)
                get_vector_store(text_chunks)
                st.success("Process completed successfully!")

if __name__ == "__main__":
    main()

In [37]:

!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501


npx: installed 22 in 3.668s
your url is: https://fancy-files-enjoy.loca.lt
^C
